In [ ]:
!pip install peft



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.5 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate


In [ ]:
# !pip freeze





In [ ]:
import os
# import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM , PreTrainedTokenizerFast, AdamW, BitsAndBytesConfig, GPT2LMHeadModel, LlamaTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import tqdm
import urllib.request
import bitsandbytes
import tqdm


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install accelerate

In [ ]:
# 모델, 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
)

#   torch_dtype='auto', low_cpu_mem_usage=True
# ).to(device='auto', non_blocking=True)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
class ChatDataset(Dataset):
    def __init__(self, train_data, tokenizer):
        self.train_data = train_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        question = self.train_data.Question.iloc[idx]
        answer = self.train_data.Answer.iloc[idx]
        bos_token = self.tokenizer.bos_token_id
        eos_token = self.tokenizer.eos_token_id

        sent = self.tokenizer.encode('<usr>' + question + '<sys>' + answer, add_special_tokens=False)
        return torch.tensor([bos_token] + sent + [eos_token], dtype=torch.long)

def collate_fn(batch):
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)

df = pd.read_csv('/content/last_df.csv')

# # 모델, 토크나이저 초기화
# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
# )
# model = AutoModelForCausalLM.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   pad_token_id=tokenizer.eos_token_id,
#   torch_dtype='auto', low_cpu_mem_usage=True
# ).to(device='auto', non_blocking=True)


# 수정된 LoRA 설정: 대상 모듈 이름을 수정함
lora_config = LoraConfig(
    r=8,  # LoRA 업데이트 행렬의 순위
    lora_alpha=32,  # LoRA 스케일링 팩터
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # 수정된 대상 모듈 이름
    lora_dropout=0.1,  # LoRA 드롭아웃 비율
    bias="none",  # LoRA의 편향 설정
    task_type="CAUSAL_LM"  # 태스크 유형
)


In [ ]:
# 모델을 LoRA 파인 튜닝에 맞게 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 여러 GPU를 사용하기 위해 DataParallel 적용
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

model.to(device)  # 모델을 GPU로 이동

batch_size = 1  # 배치 크기를 1로 설정합니다.
EPOCHS = 10
steps = len(df) // batch_size + 1
PATIENCE = 2  # 성능 개선이 없는 경우, 2 에포크 이후 학습 중단
best_loss = float('inf')
patience_counter = 0

# 데이터셋과 데이터 로더 준비
chat_dataset = ChatDataset(df, tokenizer)
data_loader = DataLoader(chat_dataset, batch_size=batch_size, collate_fn=collate_fn)


# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, eps=1e-08)
for epoch in range(EPOCHS):
    model.train()  # 모델을 학습 모드로 설정
    epoch_loss = 0

    for batch in tqdm.tqdm(data_loader):
        # 배치를 직접 장치로 이동
        batch = batch.to(device)
        labels = batch.clone()

        optimizer.zero_grad()

        # 모델의 입력과 레이블을 설정
        # 이 경우, 입력과 레이블이 같은 텐서를 사용합니다
        result = model(input_ids=batch, labels=labels)
        loss = result.loss
        batch_loss = loss.mean()

        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss.item()

    epoch_loss /= len(data_loader)

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

    # 얼리 스톱 조건 검사
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0  # 성능 개선 시 카운터 초기화
    else:
        patience_counter += 1  # 성능 개선 없음, 카운터 증가

    # Save the model and tokenizer after each epoch
    model_save_path = f'model_directory/model_epoch_{epoch + 1}'
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    if patience_counter >= PATIENCE:
        print("Early stopping triggered. Training stopped.")
        break


100%|██████████| 7430/7430 [27:45<00:00,  4.46it/s]


[Epoch:    1] cost = 1.64614868


100%|██████████| 7430/7430 [27:43<00:00,  4.47it/s]


[Epoch:    2] cost = 1.42640152


100%|██████████| 7430/7430 [27:46<00:00,  4.46it/s]


...

[Epoch:    8] cost = 0.70390152


100%|██████████| 7430/7430 [27:49<00:00,  4.45it/s]


[Epoch:    9] cost = 0.641806539


100%|██████████| 7430/7430 [27:47<00:00,  4.45it/s]


[Epoch:   10] cost = 0.585844658


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 구글 드라이브에 모델 저장
model.save_pretrained('/content/drive/MyDrive/DBDBDeep/model_KAKAO')
tokenizer.save_pretrained('/content/drive/MyDrive/DBDBDeep/model_KAKAO')

('/content/drive/MyDrive/DBDBDeep/model_KAKAO/tokenizer_config.json',
 '/content/drive/MyDrive/DBDBDeep/model_KAKAO/special_tokens_map.json',
 '/content/drive/MyDrive/DBDBDeep/model_KAKAO/tokenizer.json')

In [ ]:
# 모델, 저장 경로 설정
model_dir='/content/drive/MyDrive/DBDBDeep/model_KAKAO'

In [ ]:
# 모델, 토크나이저 초기화
model = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)



Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
text = '벽지가 벌어졌어'

In [ ]:
sent = '<usr>' + text + '<sys>'

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)

In [ ]:
output = model.generate(input_ids, max_length=100, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].tolist())

In [ ]:
output = model.generate(input_ids, max_length=100, do_sample=True, top_k=10)
tokenizer.decode(output[0].tolist())

'[BOS]<usr>벽지가 벌어졌어<sys>벽지가 벌어지면 주변 구조물에도 영향을 미칠까요?<sys>벽지면이 벌어지면 주변 구조물도 피해를 받을 수 있습니다. 특히, 지지대와 창문 틀 등을 고정시키는 부분이 피해를 입을 수 있습니다. 이러한 경우에는 주변 구조물의 안전을 보장하기 위해 조치가 필요할 수 있습니다.[EOS]'

In [ ]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

In [ ]:
# 사용자 입력 후 응답하는 챗봇 함수 생성

def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)   # 입력 문장을 인코딩
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # 텐서로 변환하고 GPU로 이동합니다.

    # 모델을 사용하여 챗봇의 응답을 생성합. 최대 길이는 50, 샘플링을 수행하며 상위 2개 토큰을 선택.
    output = model.generate(input_ids, max_length=150, do_sample=True, top_k=2)
    sentence = tokenizer.decode(output[0].tolist())  # 토큰을 문장으로 디코딩
    chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
    return chatbot_response

In [ ]:
user_text=input('질문을 입력하세요: ')

질문을 입력하세요: 도배비용 알려줘


In [ ]:
return_answer_by_chatbot(user_text)

In [ ]:
return_answer_by_chatbot('도배할때 주의사항이 뭐야')